# Using the Ansible builtin URI method to query a Redfish service


Version 0.14

## Introduction

This Jupyter Notebook uses the Ansible builtin `uri` method for getting and setting parameters in the Redfish service of an iLO 5.

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.1) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}
let ilo5Port=443

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

ilo5IP="iLO5SimulatorIP"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
#alias ResetIlo5Simulator="ssh rstsimul@${iLO5SimulatorIP} $Id $iLO5SimulatorBasePort"
w=$(basename $PWD)
alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create Ansible inventory file
cat >> hosts << __EOF__
[OneViewManagedBmcs]
ilo5simulators ansible_port={{ ILOSYBASEPORT }}

__EOF__

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetIlo5Simulator 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and Set Redfish properties using Ansible built-in uri module 

The following cells retrieves properties from hosts listed in the [hosts](hosts) file using the [ansible.builtin.uri](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/uri_module.html) module.

### GET SSD Wear status

In [ ]:
ansible-playbook -i hosts 2/GetWearStatusUsingBuiltInUri.yml

### Change IndicatorLED status

TBD

In [ ]:
ansible-playbook -i hosts 2/SetIndicatorLEDUsingBuilInUri.yml

## Summary

In this workshop, you TBD Go to the [Conclusion Notebook](3-Conclusion.ipynb) if you are finished.